In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Vitamin_D_Levels"
cohort = "GSE118723"

# Input paths
in_trait_dir = "../../input/GEO/Vitamin_D_Levels"
in_cohort_dir = "../../input/GEO/Vitamin_D_Levels/GSE118723"

# Output paths
out_data_file = "../../output/preprocess/Vitamin_D_Levels/GSE118723.csv"
out_gene_data_file = "../../output/preprocess/Vitamin_D_Levels/gene_data/GSE118723.csv"
out_clinical_data_file = "../../output/preprocess/Vitamin_D_Levels/clinical_data/GSE118723.csv"
json_path = "../../output/preprocess/Vitamin_D_Levels/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import numpy as np
import os
import re

# 1. Determine if gene expression data is available
# Based on the background information, this study collected single-cell RNA-seq data
# which contains gene expression information, so gene expression data is available.
is_gene_available = True

# 2. Variable availability and data type conversion
# 2.1 Identify keys for trait, age, and gender

# For trait (Vitamin D Levels): 
# Looking at the sample characteristics, there's no direct mention of Vitamin D levels
# The study focuses on induced pluripotent stem cells and gene expression variation
trait_row = None  # No Vitamin D data available

# For age:
# No age information is provided in the sample characteristics
age_row = None

# For gender:
# No gender information is provided in the sample characteristics
gender_row = None

# 2.2 Define conversion functions
def convert_trait(value):
    """
    Convert trait values to appropriate data type.
    Since trait data is not available, this function is just a placeholder.
    """
    if value is None or pd.isna(value):
        return None
    
    # Extract the value after the colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        # Convert to float assuming it's continuous
        return float(value)
    except (ValueError, TypeError):
        return None

# Age conversion function (not used but defined for completeness)
def convert_age(value):
    """
    Convert age values to appropriate data type.
    Since age data is not available, this function is just a placeholder.
    """
    if value is None or pd.isna(value):
        return None
    
    # Extract the value after the colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        # Convert to float assuming age is continuous
        return float(value)
    except (ValueError, TypeError):
        return None

# Gender conversion function (not used but defined for completeness)
def convert_gender(value):
    """
    Convert gender values to appropriate data type.
    Since gender data is not available, this function is just a placeholder.
    """
    if value is None or pd.isna(value):
        return None
    
    # Extract the value after the colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    # Binary coding: female=0, male=1
    if 'female' in value:
        return 0
    elif 'male' in value:
        return 1
    else:
        return None

# 3. Save metadata
# Determine trait availability
is_trait_available = trait_row is not None

# Save initial filtering information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip the clinical feature extraction step


### Step 3: Gene Data Extraction

In [ ]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# First check how many lines we have after the header to understand the data structure
import gzip
marker_found = False
with gzip.open(matrix_file_path, 'rt') as file:
    for i, line in enumerate(file):
        if "!series_matrix_table_begin" in line:
            marker_found = True
            print(f"Found marker at line {i}")
            
            # Read the header line
            header_line = next(file)
            print(f"Header line contains {len(header_line.strip().split('\t'))} columns")
            
            # Try to read the next 3 lines to see the data structure
            print("\nFirst 3 data rows after header:")
            for j in range(3):
                next_line = next(file, None)
                if next_line:
                    cols = next_line.strip().split('\t')
                    print(f"Row {j+1}: ID={cols[0]}, {len(cols)} columns total")
                    if j == 0:  # Print a few sample values from first row to verify data format
                        print(f"Sample values: {cols[1:5]}...")
                else:
                    print(f"No data at row {j+1}")
            break

# Attempt to read data with pandas directly
try:
    # Based on the debugging, we'll read the file directly focusing on the data part
    with gzip.open(matrix_file_path, 'rt') as file:
        # Skip until we find the marker
        marker_line_num = 0
        for i, line in enumerate(file):
            if "!series_matrix_table_begin" in line:
                marker_line_num = i
                break
        
    # Now read the data with pandas, skipping to right after the marker line
    gene_data = pd.read_csv(
        matrix_file_path, 
        compression='gzip', 
        skiprows=marker_line_num+1,  # +1 to skip the marker itself
        sep='\t', 
        index_col=0,
        low_memory=False,  # Avoid mixed type inference errors
        header=0,  # First row is header
        nrows=50,  # Read just a subset first to diagnose
        comment='!',  # Skip lines starting with !
        quotechar='"'  # Handle quotes properly
    )
    
    # Check if we have data
    if gene_data.shape[0] > 0:
        print(f"\nSuccessfully read gene data: {gene_data.shape} (showing first 50 rows)")
        print(f"First 5 row IDs (gene/probe identifiers):")
        print(gene_data.index[:5])
    else:
        print("No data rows found in gene expression matrix")
    
    # If all went well, now read the entire dataset
    if gene_data.shape[0] > 0:
        gene_data = pd.read_csv(
            matrix_file_path, 
            compression='gzip', 
            skiprows=marker_line_num+1,
            sep='\t', 
            index_col=0,
            low_memory=False,
            header=0,
            comment='!',
            quotechar='"'
        )
        print(f"\nFull gene expression data shape: {gene_data.shape}")
    
    # Save the gene expression data to a file
    if gene_data.shape[0] > 0:
        os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
        gene_data.to_csv(out_gene_data_file)
        print(f"Gene expression data saved to {out_gene_data_file}")
        
except Exception as e:
    print(f"Error reading gene data: {e}")
    gene_data = pd.DataFrame()  # Create empty DataFrame in case of failure

# If still unsuccessful, display the file structure in more detail
if gene_data.shape[0] == 0:
    print("\nDetailed inspection of file structure:")
    with gzip.open(matrix_file_path, 'rt') as file:
        # Skip to the marker
        for line in file:
            if "!series_matrix_table_begin" in line:
                break
        
        # Read the header
        header = next(file).strip()
        print(f"Header: {header[:100]}...")
        
        # Try to read the first 5 data lines and print raw content
        print("\nRaw content of first 5 data lines:")
        for i in range(5):
            line = next(file, None)
            if line:
                print(f"Line {i+1} (first 100 chars): {line.strip()[:100]}...")
                print(f"Line {i+1} length: {len(line)}")
            else:
                print(f"No content at line {i+1}")


### Step 4: Gene Data Extraction

In [ ]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Detailed inspection of the file structure
print("Examining file structure to locate gene expression data:")
section_markers = {
    "series_begin": 0,
    "series_end": 0,
    "platform_begin": 0,
    "platform_end": 0,
    "series_matrix_begin": 0,
    "series_matrix_end": 0
}

with gzip.open(matrix_file_path, 'rt') as file:
    for i, line in enumerate(file):
        if i < 100 or i % 1000 == 0:  # Print first 100 lines and then every 1000th line
            print(f"Line {i}: {line[:50].strip()}...")
        
        # Track section markers
        if "!Series_table_begin" in line:
            section_markers["series_begin"] = i
        elif "!Series_table_end" in line:
            section_markers["series_end"] = i
        elif "!Platform_table_begin" in line:
            section_markers["platform_begin"] = i
        elif "!Platform_table_end" in line:
            section_markers["platform_end"] = i
        elif "!series_matrix_table_begin" in line:
            section_markers["series_matrix_begin"] = i
        elif "!series_matrix_table_end" in line:
            section_markers["series_matrix_end"] = i
            
        # Stop after we've found all markers or read enough of the file
        if all(v > 0 for v in section_markers.values()) or i > 10000:
            break

print("\nSection markers found:")
for marker, line_num in section_markers.items():
    print(f"{marker}: line {line_num}")

# Try reading from the SOFT file instead, which might contain the gene data
print("\nAttempting to read gene data from SOFT file:")
try:
    # First inspect the SOFT file structure
    with gzip.open(soft_file_path, 'rt') as file:
        for i, line in enumerate(file):
            if i < 20:  # Print first 20 lines
                print(f"SOFT file line {i}: {line[:50].strip()}...")
            if i >= 20:
                break
    
    # Try to extract gene annotations from the SOFT file
    gene_metadata = get_gene_annotation(soft_file_path)
    print(f"\nGene metadata shape: {gene_metadata.shape}")
    
    if gene_metadata.shape[0] > 0:
        print("First 5 rows of gene metadata:")
        print(gene_metadata.head())
        
        # Check if this contains expression data
        if any('value' in col.lower() or 'expr' in col.lower() for col in gene_metadata.columns):
            print("\nFound potential expression data in gene metadata")
            gene_data = gene_metadata
        else:
            print("\nNo expression data columns found in gene metadata")
            gene_data = pd.DataFrame()
    else:
        print("No gene metadata found in SOFT file")
        gene_data = pd.DataFrame()
    
    # If we found data, try to identify expression columns and save them
    if gene_data.shape[0] > 0 and gene_data.shape[1] > 1:
        if 'ID' in gene_data.columns:
            gene_data = gene_data.set_index('ID')
        
        # Print the first 20 row IDs
        print("\nFirst 20 gene/probe identifiers:")
        print(gene_data.index[:20])
        
        # Print shape to understand the dataset dimensions
        print(f"\nGene expression data shape: {gene_data.shape}")
        
        # Save the gene data
        os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
        gene_data.to_csv(out_gene_data_file)
        print(f"Gene data saved to {out_gene_data_file}")
    else:
        print("\nNo usable gene expression data found in either file")
        
except Exception as e:
    print(f"Error extracting data from SOFT file: {e}")
    gene_data = pd.DataFrame()

# If we still don't have any gene data, check for other files in the directory
if gene_data.shape[0] == 0:
    print("\nSearching for alternative data files in the directory:")
    all_files = os.listdir(in_cohort_dir)
    for file in all_files:
        print(f"  {file}")
    
    print("\nThis dataset may require accessing supplementary files or raw data from GEO.")
    print("The gene expression data doesn't appear to be in the standard format.")


### Step 5: Gene Identifier Review

In [ ]:
# Based on the provided information, I need to determine if gene identifier mapping is required.
# The output shows the SOFT file structure, but there's no clear view of the gene identifiers.
# The GEO dataset GSE118723 appears to be RNA-Seq data from human iPSCs.

# Let's try to find and examine the supplementary files which might contain the gene expression data
import os
import gzip
import pandas as pd

# List all files in the directory
print("Available files in the directory:")
for file in os.listdir(in_cohort_dir):
    print(file)

# Check for expression data in supplementary files
supp_files = [f for f in os.listdir(in_cohort_dir) if 'genes' in f.lower() or 'count' in f.lower() or 'expr' in f.lower()]
if supp_files:
    print("\nExamining potential gene expression files:")
    for file in supp_files:
        file_path = os.path.join(in_cohort_dir, file)
        print(f"\nFile: {file}")
        
        # Try to read the file and examine the first few lines
        try:
            if file.endswith('.gz'):
                with gzip.open(file_path, 'rt') as f:
                    for i, line in enumerate(f):
                        if i < 5:  # Print only first 5 lines
                            print(f"Line {i}: {line.strip()}")
                        else:
                            break
            else:
                with open(file_path, 'r') as f:
                    for i, line in enumerate(f):
                        if i < 5:  # Print only first 5 lines
                            print(f"Line {i}: {line.strip()}")
                        else:
                            break
        except Exception as e:
            print(f"Error reading file: {e}")

# Since we couldn't get a clear view of the gene identifiers from the available information,
# I'll need to make an informed decision based on the dataset description
# From the dataset title "Discovery and characterization of vitamin D sensitive genes in human induced pluripotent stem cells"
# and knowing it's RNA-Seq data processed with tools like featureCounts, the gene identifiers are likely Ensembl IDs or gene symbols.

# For RNA-Seq data processed with standard pipelines (Subread/featureCounts), Ensembl gene IDs are commonly used
# Therefore, it's likely that gene mapping may be required if the IDs are not human gene symbols

requires_gene_mapping = True


### Step 6: Gene Annotation

### Step 7: Gene Annotation

In [ ]:
# Initialize variables outside of try block
# Get the file paths using the library function
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)
platform_info = []
gene_data = pd.DataFrame()  # Empty DataFrame in case it wasn't defined in previous steps

try:
    # Extract gene annotation using the library function
    print("Extracting gene annotation from SOFT file...")
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # Display column names of the annotation dataframe
    print("\nGene annotation columns:")
    print(gene_annotation.columns.tolist())
    
    # Preview the annotation dataframe
    print("\nGene annotation preview:")
    annotation_preview = preview_df(gene_annotation)
    print(annotation_preview)
    
    # Check if we have annotation data
    if gene_annotation.shape[0] > 0:
        # Analyze the SOFT file to identify the data type and mapping information
        with gzip.open(soft_file_path, 'rt', encoding='latin-1') as f:
            for line in f:
                if line.startswith("!Platform_title") or line.startswith("!Series_title") or "description" in line.lower():
                    platform_info.append(line.strip())
        
        print("\nPlatform information:")
        for line in platform_info:
            print(line)
        
        # Look for gene symbol column
        gene_symbol_candidates = [col for col in gene_annotation.columns if 'gene' in col.lower() or 'symbol' in col.lower() or 'name' in col.lower()]
        print(f"\nPotential gene symbol columns: {gene_symbol_candidates}")
    else:
        print("No gene annotation found in the SOFT file.")

except Exception as e:
    print(f"Error analyzing gene annotation data: {e}")
    gene_annotation = pd.DataFrame()

# Based on platform_info, determine if this is really gene expression data
is_gene_expression = False
for info in platform_info:
    if 'expression' in info.lower() or 'transcript' in info.lower() or 'mrna' in info.lower():
        is_gene_expression = True
        break

print(f"\nIs this dataset likely to contain gene expression data based on platform info? {is_gene_expression}")

# If this isn't gene expression data, we need to update our metadata
if not is_gene_expression and platform_info:  # Only update if we found platform info
    print("\nNOTE: Based on our analysis, this dataset doesn't appear to contain gene expression data.")
    print("It appears to be a different type of data (possibly SNP array or other genomic data).")
    # Update is_gene_available for metadata
    is_gene_available = False
    
    # Save the updated metadata
    validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )